## Actualización GitHub

git add .
git commit -m "Actualizo documentación"
git push


## Importación de librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

##  1. CARGA DEL DATASET + REVISIÓN INICIAL

In [2]:
ruta= "C:/Users/lamas/Documents/Ejercicios/Prácticas/bmw_pricing_v3 (1).csv"

In [3]:
df_bmw= pd.read_csv(ruta)

In [4]:
df_bmw.head()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,2018-01-01
1,BMW,M4,13929.0,317.0,NaN,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2018-02-01
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,False,NaN,True,False,True,False,10200.0,2018-02-01
3,BMW,420,128035.0,135.0,NaN,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2018-02-01
4,BMW,425,97097.0,160.0,NaN,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,2018-04-01


In [5]:
df_bmw.tail()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
4838,BMW,218 Gran Tourer,39743.0,110.0,NaN,diesel,black,NaN,False,True,False,NaN,False,False,True,False,14600.0,2018-08-01
4839,BMW,218 Active Tourer,49832.0,100.0,2015-06-01,diesel,grey,NaN,False,True,False,NaN,False,False,True,True,17500.0,2018-08-01
4840,BMW,218 Gran Tourer,19633.0,110.0,2015-10-01,diesel,grey,van,False,True,False,NaN,False,False,True,True,17000.0,2018-09-01
4841,BMW,218 Active Tourer,27920.0,110.0,2016-04-01,diesel,brown,van,True,True,False,False,False,False,True,True,22700.0,2018-09-01
4842,BMW,525,195840.0,160.0,NaN,diesel,grey,NaN,True,NaN,False,NaN,True,False,True,True,15500.0,2018-09-01


In [6]:
df_bmw.shape

(4843, 18)

In [7]:
df_bmw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   marca                        3873 non-null   object 
 1   modelo                       4840 non-null   object 
 2   km                           4841 non-null   float64
 3   potencia                     4842 non-null   float64
 4   fecha_registro               2420 non-null   object 
 5   tipo_gasolina                4838 non-null   object 
 6   color                        4398 non-null   object 
 7   tipo_coche                   3383 non-null   object 
 8   volante_regulable            4839 non-null   object 
 9   aire_acondicionado           4357 non-null   object 
 10  camara_trasera               4841 non-null   object 
 11  asientos_traseros_plegables  1452 non-null   object 
 12  elevalunas_electrico         4841 non-null   object 
 13  bluetooth         

In [8]:
df_bmw.describe(include="all")

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
count,3873,4840,4.841000e+03,4842.000000,2420,4838,4398,3383,4839,4357,4841,1452,4841,4115,4843,4115,4837.000000,4842
unique,1,76,NaN,NaN,180,5,10,8,2,2,2,2,2,2,2,2,NaN,13
top,BMW,320,NaN,NaN,2013-07-01,diesel,black,estate,True,True,False,False,False,False,True,True,NaN,2018-05-01
freq,3873,752,NaN,NaN,93,4631,1499,1107,2662,3465,3864,1150,2612,3119,4514,2240,NaN,809
mean,NaN,NaN,1.409593e+05,128.981826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15831.920612,NaN
std,NaN,NaN,6.020853e+04,38.994839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9222.630708,NaN
min,NaN,NaN,-6.400000e+01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,NaN
25%,NaN,NaN,1.028840e+05,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10800.000000,NaN
50%,NaN,NaN,1.410800e+05,120.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14200.000000,NaN
75%,NaN,NaN,1.752170e+05,135.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18600.000000,NaN


# Pregunta 1

In [19]:
#Detección de nulos
(df_bmw.isnull().sum()/len(df_bmw))*100

marca                    20.028908
modelo                    0.061945
km                        0.041297
potencia                  0.020648
tipo_gasolina             0.103242
color                     9.188520
tipo_coche               30.146603
volante_regulable         0.082593
aire_acondicionado       10.035102
camara_trasera            0.041297
elevalunas_electrico      0.041297
bluetooth                15.032005
gps                       0.000000
alerta_lim_velocidad     15.032005
precio                    0.123890
fecha_venta               0.020648
capacidad_deposito      100.000000
dtype: float64

En este caso, observamos que la columna "asientos_traseros_plegables" tiene un porcentaje altísimo de nulos (70%), al igual que la "fecha_registro" que tiene un 50% de valores nulos. Vamos a eliminar ambas generando una nueva variable agrupe las variables con un porcentaje de nulos mayor al 50%.

In [10]:
cols = df_bmw.columns[df_bmw.isnull().mean() > 0.5]
cols

Index(['fecha_registro', 'asientos_traseros_plegables'], dtype='object')

In [11]:
#Eliminamos
df_bmw.drop(columns=cols, inplace=True)

In [12]:
#Revisar valores incoherentes
df_bmw.loc[df_bmw["tipo_gasolina"] == "Eléctrico", "capacidad_deposito"] = 0

In [17]:
df_bmw[df_bmw.marca.isna()]

,marca,modelo,km,potencia,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta,capacidad_deposito
0,NaN,118,140411.0,100.0,diesel,black,NaN,True,True,False,True,NaN,True,NaN,11300.0,2018-01-01,NaN
8,NaN,Z4,123886.0,125.0,petrol,black,convertible,True,False,False,False,NaN,True,False,6200.0,2018-03-01,NaN
11,NaN,325,228000.0,145.0,diesel,black,NaN,True,True,False,True,False,True,NaN,13300.0,2018-03-01,NaN
18,NaN,325,205474.0,145.0,diesel,NaN,NaN,True,True,False,True,NaN,True,NaN,11000.0,2018-04-01,NaN
21,NaN,218,24868.0,100.0,petrol,grey,convertible,False,True,False,False,NaN,True,True,31000.0,2018-05-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,NaN,220 Active Tourer,153358.0,140.0,diesel,grey,van,False,NaN,False,False,False,True,True,11700.0,2018-02-01,NaN
4807,NaN,218 Active Tourer,197276.0,112.0,diesel,white,van,True,True,False,False,NaN,True,True,13100.0,2018-02-01,NaN
4825,NaN,225 Active Tourer,33300.0,170.0,petrol,NaN,van,True,True,False,False,False,False,True,19200.0,2018-05-01,NaN
4834,NaN,218 Gran Tourer,47782.0,110.0,diesel,blue,NaN,False,NaN,False,False,False,True,True,14900.0,2018-07-01,NaN


In [21]:
#Eliminar nulos del target
df_bmw["precio"].dropna(inplace=True)


In [22]:
(df_bmw.isnull().sum()/len(df_bmw))*100

marca                    20.028908
modelo                    0.061945
km                        0.041297
potencia                  0.020648
tipo_gasolina             0.103242
color                     9.188520
tipo_coche               30.146603
volante_regulable         0.082593
aire_acondicionado       10.035102
camara_trasera            0.041297
elevalunas_electrico      0.041297
bluetooth                15.032005
gps                       0.000000
alerta_lim_velocidad     15.032005
precio                    0.123890
fecha_venta               0.020648
capacidad_deposito      100.000000
dtype: float64